# Check Product Propensity Customer Table
* StelllarAlgo Data Science
* Ryan Kazmerik
* Oct 21, 2022

In [43]:
import boto3
import pandas as pd

from shared_utilities import helpers

In [44]:
session = helpers.get_aws_session("Stellaralgo-DataScienceAdmin")

Authorized as AROASQ4JELIXYLYV6P4UV:rkazmerik@stellaralgo.com


In [45]:
CLUSTER = "prod-app"
DATABASE = "stlrcanucks"
LKUPCLIENTID = 7

CNXN = helpers.get_redshift_connection(CLUSTER, DATABASE)

Authorized as AROASQ4JELIXYLYV6P4UV:rkazmerik@stellaralgo.com


In [46]:
sql= f"""
    SELECT * FROM {DATABASE}.dw.customerproductpropensityscores
    WHERE scoredate = (
        SELECT MAX(scoredate) FROM {DATABASE}.dw.customerproductpropensityscores
    )
    AND lkupclientid = {LKUPCLIENTID}
"""

try:
    cursor = CNXN.cursor()
    cursor.execute(sql)
    data = cursor.fetchall()

    cols = [row[0] for row in cursor.description]
    df_scores = pd.DataFrame(data=data, columns=cols)

    print(f"DATABASE: {DATABASE}")
    print(f"ROWS RETRIEVED: {df_scores.shape[0]}")
    print(f"MAX SCORE DATE: {df_scores['scoredate'].max()}")

except Exception as e:
    print(f"ERROR:, {e}")
    cursor.execute("ROLLBACK")

finally:
    cursor.close()


DATABASE: stlrcanucks
ROWS RETRIEVED: 2213
MAX SCORE DATE: 2022-10-21 14:14:33


In [47]:
df_scores.groupby(["product","product_current"]).agg({
    "product": "count",
})

product
product     product_current         
Any         Individual          2000
Full Season Half Season          102
            Mini Pack             11
            Quarter               53
Half Season Mini Pack              3
            Quarter               29
Quarter     Mini Pack             15

### Done